<a href="https://colab.research.google.com/github/luciaeveberger/data_science_final_project/blob/main/Kaggle10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import neighbors
from sklearn.preprocessing import MinMaxScaler
from keras.regularizers import l2
from tensorflow.keras import regularizers
from keras.layers import Dropout
import tensorflow as tf
from sklearn.svm import SVR
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import dateutil

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def pre_process(data):
  # User Name column is not relevent
  data.drop('User Name', 1,inplace=True)

  # Replacing null with 0 and non-null with 1
  data['Personal URL'] = data['Personal URL'].notnull().astype('int')

  # Set --> 1 and not Set --> 0
  data['Profile Cover Image Status'] = np.where(data['Profile Cover Image Status']=='Set', 1, 0)

  # One-Hot encoding for Profile Verification Status column
  #data = pd.get_dummies(data, columns = ['Profile Verification Status'])


  #Frequnce encoding for Profile Verification Status column
  enc_prof_verif = (data.groupby('Profile Verification Status').size()) / len(data)
  data['Profile Verification Status'] = data['Profile Verification Status'].apply(lambda x : enc_prof_verif[x])

  #Frequnce encoding for Color columns
  data['Profile Theme Color'].fillna(0,inplace = True)
  enc_color_theme = (data.groupby('Profile Theme Color').size()) / len(data)
  data['Profile Theme Color'] = data['Profile Theme Color'].apply(lambda x : enc_color_theme[x])
  data['Profile Text Color'].fillna(0,inplace = True)
  enc_color_text = (data.groupby('Profile Text Color').size()) / len(data)
  data['Profile Text Color'] = data['Profile Text Color'].apply(lambda x : enc_color_text[x])
  data['Profile Page Color'].fillna(0,inplace = True)
  enc_color_page = (data.groupby('Profile Page Color').size()) / len(data)
  data['Profile Page Color'] = data['Profile Page Color'].apply(lambda x : enc_color_page[x])

  # Mapping True/False --> 1/0
  data['Is Profile View Size Customized?'] = data['Is Profile View Size Customized?'].astype(int)

  # Dropping irrelevant Cols (Maybe we should use them)
  data.drop(['UTC Offset', 'Location'], axis=1, inplace=True)

  # Enabled --> 1 and Disabled or ??? --> 0
  data['Location Public Visibility'] = np.where(data['Location Public Visibility'].str.lower() == 'enabled', 1, 0)

  # Frequency encoding for user language
  enc_user_lang = (data.groupby('User Language').size()) / len(data)
  data['User Language'] = data['User Language'].apply(lambda x : enc_user_lang[x])

  # Frequency encoding for User time zone
  data['User Time Zone'].fillna(data['User Time Zone'].value_counts().index[0],inplace = True)
  enc_time_zone = (data.groupby('User Time Zone').size()) / len(data)
  data['User Time Zone'] = data['User Time Zone'].apply(lambda x : enc_time_zone[x])

  # Replacing the white spaces with nan
  data['Profile Category'] = data['Profile Category'].replace(r'^\s*$', np.nan, regex=True)
  # Replacing nan with 'unknown'
  data['Profile Category'].fillna('unknown',inplace = True)

  # One-hot encoding
  #data = pd.get_dummies(data, columns = ['Profile Category'])

  # Frequency encoding for profile category
  enc_prof_cat = (data.groupby('Profile Category').size()) / len(data)
  data['Profile Category'] = data['Profile Category'].apply(lambda x : enc_prof_cat[x])


  # Median imputation for 'Avg Daily Profile Clicks'And'Avg Daily Profile Visit Duration in seconds'
  imp_median = SimpleImputer( strategy='median') #for median imputation replace 'mean' with 'median'
  imp_median.fit(data[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']])
  imputed_train_df = imp_median.transform(data[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']])
  imputed_train_df = pd.DataFrame(imputed_train_df,columns=['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds'])
  data[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']] = imputed_train_df[['Avg Daily Profile Clicks','Avg Daily Profile Visit Duration in seconds']]

  # Removing Outliers
  # Q1 = data['Num of Followers'].quantile(0.25)
  # Q3 = data['Num of Followers'].quantile(0.75)
  # IQR = Q3 - Q1
  # Q3 + 1.5 * IQR
  # data = data[data['Num of Followers'] <(Q3 + 1.5 * IQR)]

  # Making Id column as index
  data.set_index('Id',inplace=True)

  # Dropping Pic Col for now
  data.drop('Profile Image', 1,inplace=True)
  # Dropping Personal Url column
  #data.drop('Personal URL', 1,inplace=True)

  #Parsing Number of years until 2020
  data['Profile Creation Timestamp'] = data['Profile Creation Timestamp'].apply(lambda x: dateutil.parser.parse(x))
  data['Profile Creation Timestamp']=pd.DatetimeIndex(data['Profile Creation Timestamp']).year
  data['Profile Creation Timestamp'] = 2020-data['Profile Creation Timestamp']

  return data




In [ ]:
df_train = pre_process(pd.read_csv("/content/gdrive/MyDrive/ift6758-a20/train.csv"))
df_test = pre_process(pd.read_csv("/content/gdrive/MyDrive/ift6758-a20/test.csv")) 

In [ ]:
y_col = ['Num of Profile Likes']
x_col = df_train.columns.difference(['Num of Profile Likes'])

In [ ]:
#Standardization - Train
df_train_stand = df_train.copy()
for i in x_col:
    
    # fit on training data column
    scale = StandardScaler().fit(df_train_stand[[i]])
    
    # transform the training data column
    df_train_stand[i] = scale.transform(df_train_stand[[i]])

In [ ]:
#Standardization - Test
df_test_stand = df_test.copy()
for i in x_col:
    
    # fit on training data column
    scale = StandardScaler().fit(df_test_stand[[i]])
    
    # transform the training data column
    df_test_stand[i] = scale.transform(df_test_stand[[i]])

In [ ]:
# pearson's correlation feature selection 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
# define feature selection
fs = SelectKBest(score_func=f_regression, k='all')
# apply feature selection
X_selected = fs.fit_transform(df_train_stand[x_col], df_train_stand[y_col].values.ravel())
names = df_train_stand[x_col].columns.values[fs.get_support()]
scores = fs.scores_[fs.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
#Sort the dataframe for better visualization
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)

                                     Feat_names    F_Scores
8                                  Personal URL  210.237773
0                      Avg Daily Profile Clicks  162.683265
9                              Profile Category  133.397593
6                       Num of People Following  115.705358
15                  Profile Verification Status   67.559534
3                    Location Public Visibility   43.336782
11                   Profile Creation Timestamp   28.172260
12                           Profile Page Color   26.618781
2              Is Profile View Size Customized?   18.128637
7                         Num of Status Updates   14.007676
13                           Profile Text Color   10.973332
17                               User Time Zone    9.846833
14                          Profile Theme Color    8.691975
16                                User Language    2.251782
1   Avg Daily Profile Visit Duration in seconds    1.257316
5                              Num of Fo

In [ ]:
num_features = df_train_stand[x_col].shape[1]
model = Sequential()
model.add(Dense(22, activation='relu', input_dim=num_features))
model.add(Dense(22, activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics='mean_squared_logarithmic_error')
model.fit(df_train_stand[x_col], df_train_stand[y_col], batch_size=4, epochs=100,validation_split=0.2)

Epoch 1/100
1500/1500 [==============================] - 3s 2ms/step - loss: 16.2414 - mean_squared_logarithmic_error: 16.2414 - val_loss: 7.5122 - val_mean_squared_logarithmic_error: 7.5122
Epoch 2/100
1500/1500 [==============================] - 2s 2ms/step - loss: 5.7116 - mean_squared_logarithmic_error: 5.7116 - val_loss: 5.4213 - val_mean_squared_logarithmic_error: 5.4213
Epoch 3/100
1500/1500 [==============================] - 2s 2ms/step - loss: 4.5328 - mean_squared_logarithmic_error: 4.5328 - val_loss: 4.8115 - val_mean_squared_logarithmic_error: 4.8115
Epoch 4/100
1500/1500 [==============================] - 2s 1ms/step - loss: 4.1625 - mean_squared_logarithmic_error: 4.1625 - val_loss: 4.5958 - val_mean_squared_logarithmic_error: 4.5958
Epoch 5/100
1500/1500 [==============================] - 2s 1ms/step - loss: 4.0192 - mean_squared_logarithmic_error: 4.0192 - val_loss: 4.4900 - val_mean_squared_logarithmic_error: 4.4900
Epoch 6/100
1500/1500 [==============================

In [ ]:
# Submission file
y_pred = model.predict(df_test_stand[x_col])
dummy = pd.DataFrame()
dummy["Id"] = df_test.index
dummy["Predicted"]=y_pred
dummy.to_csv("submission.csv", sep=',' ,index=False)
len(y_pred)